In [48]:
import pandas as pd
import numpy as np
from pandasql import sqldf
from IPython.display import clear_output

## Importing Data

In [49]:
# Add column with service name to each DataFrame
amazon = pd.read_csv("data/amazon_clean.csv")
disney = pd.read_csv("data/disney_clean.csv")
hbo = pd.read_csv("data/hbo_clean.csv")
hulu = pd.read_csv("data/hulu_clean.csv")
netflix = pd.read_csv("data/netflix_clean.csv")

In [50]:
raw_dfs = [amazon,disney,hbo,hulu,netflix]

## Initial Restriction and Combining of Data

In [51]:
# Combine all DataFrames
services = raw_dfs[0].copy()
for df in raw_dfs[1:]:
    services = sqldf("SELECT * FROM services UNION SELECT * FROM df")

In [52]:
services.sample(10)

,title,release_year,type,rating,service
8459,Gridlocked,2015,Movie,R,netflix
4741,Comicstaan Semma Comedy Pa,2020,TV Show,PG-13,amazon
9361,Hotel Transylvania,2012,Movie,PG,hulu
23051,Tourgasm,2006,TV Show,None,hbo
23938,W/O Ram,2018,Movie,R,netflix
19871,The Client,1994,Movie,PG-13,hbo
15237,Pearl,2020,Movie,R,amazon
1884,Arifureta: From Commonplace to World's Strongest,2019,TV Show,R,hulu
15694,Power Rangers Dino Charge,2015,TV Show,PG,netflix
16519,RiffTrax Presents High School Hero,2019,Movie,PG-13,amazon


In [53]:
# Separate DataFrames based on content type 
movies = sqldf("SELECT * FROM services WHERE type == \"Movie\"")
tv = sqldf("SELECT * FROM services WHERE type == \"TV Show\"")

## Merging with TMDb API

In [54]:
# Import and initial API setup
from tmdbv3api import TMDb, Search, Movie

tmdb = TMDb()
tmdb.api_key = '93854ffff857e6d378b356f9caea9972'

In [55]:
# Search the TMDb API for a given film
def get_TMDb_id(title, release_year):
    search = Search()
    # Check first 3 pages. If not found by then, move on to the next title
    for i in range(1,3):
        results = search.movies({"query": title, "page": i})
        for result in results:
            try:
                # If the release years are the same, it is likely that the instances are the same
                if(str(release_year) == result["release_date"].split("-")[0]):
                    return result["id"]
            except:
                pass
    return None

In [56]:
# Get other information for a movie based on TMDb id
def get_TMDb_info(tmdb_id=None, requested_info = []):
    # If there is no provided id, return None for all requested data
    if(tmdb_id == None):
        return [None]*len(requested_info)
    
    # Get the details of the specified title
    movie = Movie().details(tmdb_id)
    
    collected_info = []
    for i in requested_info:
        # Genres must be combined into string
        if(i == "genres"):
            genre_string = ""
            for genre in movie[i]:
                genre_string += genre["name"] + ","
            collected_info.append(genre_string[:-1])
        else:
            collected_info.append(movie[i])
    return collected_info

In [84]:
# Add columns to movies from TMDb API
ids = []
try:
    # Try to import data. If it is imported, drop rows that do not have provided IDs
    movies_imported = pd.read_csv("data/modified/movies_api_merged.csv")
    
    # Allow for re-running. Would want this if more data is requested. 
    ans = input("File found. Do you want to re-run this code? (y/n): ")
    if(ans == "y"):
        # Allow for full restarting - wipes previously calculated IDs
        ans = input("Do you want to fully restart? Warning: This will take a long time to compute (y/n): ")
        if(ans == "n"):
            movies = movies_imported.copy()
            movies = movies.dropna(subset=["tmdb_id"])
            movies = movies.reset_index(drop=True)
            # Store IDs to speed up computation if re-running
            ids = list(movies["tmdb_id"])
        # Raise an exception so that code can be re-run
        raise Exception("Re-running")
    else:
        ans = input("Do you wish to load saved data? (y/n): ")
        if(ans == "y"):
            movies = movies_imported.copy()
# If data is not found or user requests re-running, code will execute
except Exception as e:
    # Check whether IDs already exist
    if(ids == []):
        ids_loaded = False
    else:
        ids_loaded = True
        
    # Data requested from API
    other_requests = ["genres", "imdb_id", "popularity", "vote_average", "vote_count", "poster_path", "budget", "revenue", "runtime"]
    
    # Dictionary of lists that contain requested information
    other = {}
    for request in other_requests:
        other.update({request: []})
    
    for index, row in movies.iterrows():
        # Output progress 
        if(index%10 == 0):
            clear_output()
            print(index/movies.shape[0])
            
        # If there are not already IDs, get IDs from API
        if(not ids_loaded):
            ids.append(get_TMDb_id(row["title"], row["release_year"]))
            other_info = get_TMDb_info(ids[-1], other_requests)
        else:
            # If the IDs exist and the current ID is valid, get other info
            if(ids[index] != None and ids[index] != np.nan):
                other_info = get_TMDb_info(ids[index], other_requests)
            # If current ID is not valid, other info is None
            else:
                other_info = [None]*len(other_requests)
        
        if(other_info != None):
            # Add new information to dictionary of info
            for other_index in range(len(other_info)):
                other[list(other.keys())[other_index]].append(other_info[other_index])
                
    # Set new IDs if necessary
    if(not ids_loaded):
        movies["tmdb_id"] = ids
    
    # Add columns for new information
    for key in other.keys():
        movies[key] = other.get(key)
        
    # Save data as .csv
    movies.to_csv("data/modified/movies_api_merged.csv", index=False)
clear_output()

In [85]:
movies.describe()

,release_year,tmdb_id,popularity,vote_average,vote_count,budget,revenue,runtime
count,18153.000000,13797.000000,13797.000000,13797.000000,13797.000000,1.379700e+04,1.379700e+04,13752.000000
mean,2007.932628,333322.253896,13.973216,5.721381,605.890121,8.628376e+06,2.695130e+07,93.003563
std,18.321856,266816.307728,67.021047,2.118846,1926.234438,2.804993e+07,1.155505e+08,34.627442
min,1915.000000,11.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
25%,2006.000000,50720.000000,2.075000,5.300000,5.000000,0.000000e+00,0.000000e+00,82.000000
50%,2015.000000,340030.000000,5.548000,6.200000,33.000000,0.000000e+00,0.000000e+00,95.000000
75%,2019.000000,537795.000000,13.661000,7.000000,254.000000,5.000000e+04,7.000000e+00,110.000000
max,2021.000000,942567.000000,6865.258000,10.000000,30989.000000,3.800000e+08,2.847246e+09,467.000000


In [86]:
# Remove rows without TMDb IDs
movies = movies.dropna(subset=["tmdb_id"])
movies = movies.reset_index(drop=True)

In [87]:
sqldf("SELECT * FROM movies ORDER BY release_year DESC, popularity DESC, vote_average DESC").head()

,title,release_year,type,rating,service,tmdb_id,genres,imdb_id,popularity,vote_average,vote_count,poster_path,budget,revenue,runtime
0,#Home,2021,Movie,PG-13,amazon,634649.0,"Action,Adventure,Science Fiction",tt10872600,6865.258,8.3,8238.0,/1g0dhYtq4irTY1GPXvft6k4YLjm.jpg,200000000.0,1.809941e+09,148.0
1,Red,2021,Movie,PG-13,netflix,512195.0,"Action,Comedy,Crime,Thriller",tt7991608,1823.157,6.8,3077.0,/wdE6ewaKZHr62bLqCn7A2DiGShm.jpg,160000000.0,1.781430e+05,117.0
2,One,2021,Movie,PG-13,amazon,811592.0,Action,tt14199590,942.826,6.7,304.0,/3OXiTjU30gWtqxmx4BU9RVp2OTv.jpg,0.0,0.000000e+00,97.0
3,Legend,2021,Movie,PG-13,amazon,566525.0,"Action,Adventure,Fantasy",tt9376612,907.507,7.8,5713.0,/1BIoJGKbXjdFDAqUEiA2VHqkK1Z.jpg,150000000.0,4.322433e+08,132.0
4,Shang-Chi and The Legend of The Ten Rings,2021,Movie,PG-13,disney,566525.0,"Action,Adventure,Fantasy",tt9376612,907.507,7.8,5713.0,/1BIoJGKbXjdFDAqUEiA2VHqkK1Z.jpg,150000000.0,4.322433e+08,132.0


## Merging with IMDb Data

In [88]:
titles_ratings = pd.read_csv("data/title.ratings.tsv.gz", sep="\t", compression="gzip")

In [89]:
titles_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1665
1,tt0000002,6.1,206
2,tt0000003,6.5,1382
3,tt0000004,6.2,123
4,tt0000005,6.2,2168
...,...,...,...
1097082,tt9916580,7.2,5
1097083,tt9916690,6.6,5
1097084,tt9916720,6.0,67
1097085,tt9916766,6.9,15


In [90]:
movies = sqldf("SELECT * FROM movies INNER JOIN titles_ratings ON movies.imdb_id = titles_ratings.tconst")

In [93]:
movies.rename(columns = {"vote_average": "tmdb_score", "vote_count": "tmdb_count", "averageRating": "imdb_score", "numVotes": "imdb_count"}, inplace=True)

In [94]:
movies

,title,release_year,type,rating,service,tmdb_id,genres,imdb_id,popularity,tmdb_score,tmdb_count,poster_path,budget,revenue,runtime,tconst,imdb_score,imdb_count
0,#Alive,2020,Movie,R,netflix,492414.0,"Horror,Thriller",tt7611352,14.867,6.3,55.0,/3KqM6l45QQVDYQgYWnQnK1jDDHa.jpg,0.0,0.0,91.0,tt7611352,5.4,450
1,#AnneFrank - Parallel Stories,2019,Movie,PG-13,netflix,610643.0,"Documentary,Drama,History",tt9850370,16.035,7.0,35.0,/hkC4yNDFmW1yQuQhtZydMeRuaAb.jpg,0.0,0.0,92.0,tt9850370,6.4,996
2,#FriendButMarried,2018,Movie,G,netflix,503352.0,"Comedy,Drama,Romance",tt8076266,2.357,6.8,9.0,/splV83B3CqMCbHUunoyaUoRUM60.jpg,0.0,0.0,102.0,tt8076266,6.9,524
3,#FriendButMarried 2,2020,Movie,G,netflix,655293.0,"Drama,Comedy,Romance",tt11640412,2.089,7.5,2.0,/pCcvdac4PbU7U1Dgce4j9VWZ72c.jpg,0.0,0.0,104.0,tt11640412,6.6,168
4,#Lucky Number,2015,Movie,None,amazon,359732.0,Comedy,tt2538204,4.036,4.8,4.0,/wA4Ig8y8ggVVzbgZHlyOm56AxVD.jpg,1000000.0,0.0,80.0,tt2538204,5.0,444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,​Goli Soda 2,2018,Movie,PG-13,netflix,507574.0,"Drama,Action",tt8011288,1.693,5.9,5.0,/AaXQosgSTtNbVFh0T039iiysOxr.jpg,0.0,0.0,130.0,tt8011288,7.3,654
12218,​Maj Rati ​​Keteki,2017,Movie,PG-13,netflix,564862.0,Drama,tt8914956,1.161,7.0,2.0,/x8RtNnvFpB2dZWo4GAlOSh7n0s5.jpg,0.0,0.0,116.0,tt8914956,7.1,29
12219,​Mayurakshi,2017,Movie,PG-13,netflix,500787.0,Drama,tt7570242,1.960,6.8,2.0,/IR6LQgrXzCIhPTn4rKw1cViY0P.jpg,0.0,0.0,102.0,tt7570242,7.1,347
12220,​​Kuch Bheege Alfaaz,2018,Movie,PG-13,netflix,527851.0,"Romance,Drama",tt7617988,1.325,7.5,6.0,/25cv2bdc4IIIcMWdFiHC0uEy0eI.jpg,0.0,0.0,116.0,tt7617988,7.5,714


In [97]:
# Calculate mean score and number of votes for each row
mean_score = []
mean_votes = []
for index, row in movies.iterrows():
    mean_score.append((row["imdb_score"] + row["tmdb_score"])/2)
    mean_votes.append((row["imdb_count"] + row["tmdb_count"])/2)

movies["mean_score"] = mean_score
movies["mean_num_votes"] = mean_votes

In [100]:
movies.to_csv("data/modified/movies_api_imdb_merged.csv", index=False)

,title,release_year,type,rating,service,tmdb_id,genres,imdb_id,popularity,tmdb_score,tmdb_count,poster_path,budget,revenue,runtime,tconst,imdb_score,imdb_count,mean_score,mean_num_votes
0,#Alive,2020,Movie,R,netflix,492414.0,"Horror,Thriller",tt7611352,14.867,6.3,55.0,/3KqM6l45QQVDYQgYWnQnK1jDDHa.jpg,0.0,0.0,91.0,tt7611352,5.4,450,5.85,252.5
1,#AnneFrank - Parallel Stories,2019,Movie,PG-13,netflix,610643.0,"Documentary,Drama,History",tt9850370,16.035,7.0,35.0,/hkC4yNDFmW1yQuQhtZydMeRuaAb.jpg,0.0,0.0,92.0,tt9850370,6.4,996,6.70,515.5
2,#FriendButMarried,2018,Movie,G,netflix,503352.0,"Comedy,Drama,Romance",tt8076266,2.357,6.8,9.0,/splV83B3CqMCbHUunoyaUoRUM60.jpg,0.0,0.0,102.0,tt8076266,6.9,524,6.85,266.5
3,#FriendButMarried 2,2020,Movie,G,netflix,655293.0,"Drama,Comedy,Romance",tt11640412,2.089,7.5,2.0,/pCcvdac4PbU7U1Dgce4j9VWZ72c.jpg,0.0,0.0,104.0,tt11640412,6.6,168,7.05,85.0
4,#Lucky Number,2015,Movie,None,amazon,359732.0,Comedy,tt2538204,4.036,4.8,4.0,/wA4Ig8y8ggVVzbgZHlyOm56AxVD.jpg,1000000.0,0.0,80.0,tt2538204,5.0,444,4.90,224.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,​Goli Soda 2,2018,Movie,PG-13,netflix,507574.0,"Drama,Action",tt8011288,1.693,5.9,5.0,/AaXQosgSTtNbVFh0T039iiysOxr.jpg,0.0,0.0,130.0,tt8011288,7.3,654,6.60,329.5
12218,​Maj Rati ​​Keteki,2017,Movie,PG-13,netflix,564862.0,Drama,tt8914956,1.161,7.0,2.0,/x8RtNnvFpB2dZWo4GAlOSh7n0s5.jpg,0.0,0.0,116.0,tt8914956,7.1,29,7.05,15.5
12219,​Mayurakshi,2017,Movie,PG-13,netflix,500787.0,Drama,tt7570242,1.960,6.8,2.0,/IR6LQgrXzCIhPTn4rKw1cViY0P.jpg,0.0,0.0,102.0,tt7570242,7.1,347,6.95,174.5
12220,​​Kuch Bheege Alfaaz,2018,Movie,PG-13,netflix,527851.0,"Romance,Drama",tt7617988,1.325,7.5,6.0,/25cv2bdc4IIIcMWdFiHC0uEy0eI.jpg,0.0,0.0,116.0,tt7617988,7.5,714,7.50,360.0


## Release Years

This will be used to recommend services based on whether they prefer new or older content.

In [17]:
# Find the mean release year of content as well as the total number of movies on each service
movie_years = sqldf("SELECT AVG(release_year) as mean_year, COUNT(*) as n_titles, service FROM movies GROUP BY service")
movie_years

,mean_year,n_titles,service
0,2003.948093,4720,amazon
1,1998.528067,962,disney
2,1994.900187,1603,hbo
3,2011.822026,1135,hulu
4,2012.858099,5377,netflix


## Genre Breakdown

This will be used to recommend services based on what genres people most enjoy.

In [18]:
# Append each instance of a genre to the service_genres DataFrame - Movies only
try:
    service_genres = pd.read_csv("data/modified/service_genres.csv")
    ans = input("File found. Do you want to re-run this code? (y/n): ")
    if(ans == "y"):
        raise Exception("Re-running")
except:
    service_genres = pd.DataFrame(columns = ["service", "type", "genre"])
    for index, row in movies.iterrows():
        try:
            # For each genre in the listed_in column, add a new row to the DataFrame with the service, content type, and genre
            for genre in row["genres"].split(","):
                if(genre != ""):
                    service_genres = service_genres.append({"service": row["service"], "type": row["type"], "genre": genre}, ignore_index=True)
        except:
            pass
    service_genres.to_csv("data/modified/service_genres.csv", index=False)
clear_output()

File found. Do you want to re-run this code? (y/n): y


In [19]:
# Count the instances of a genre for each service and type
service_genres_counted = sqldf("SELECT service, type, genre, COUNT(*) as count FROM service_genres GROUP BY service, type, genre")

In [20]:
# Separate movie genres from TV genres
service_genres_movies = sqldf("SELECT * FROM service_genres_counted WHERE type == \"Movie\"")

In [21]:
# Get genre counts as percentage of total movies
total_count = []
percentages = []
for index, row in service_genres_movies.iterrows():
    total_count.append(np.sum(sqldf(f"SELECT count FROM service_genres_movies WHERE service == \"{row.service}\""))[0])
    percentages.append(row["count"]/total_count[-1])
    

service_genres_movies["total_on_service"] = total_count
service_genres_movies["percentage_of_total"] = percentages

In [22]:
# Sort each service's genres by count
service_genres_movies = sqldf("SELECT * FROM service_genres_movies ORDER BY service, count DESC")
# Save counted genres as .csv
service_genres_movies.to_csv("data/modified/service_genres_counted.csv", index=False)

### More Genre Exploration

Taking a look at distributions of genres and what services have the highest proportion of each genre. 

In [23]:
# Get the top 5 genres on each service
limit = 5
frames = []
# For each service, get `limit` most frequent genres
for service in np.unique(service_genres_movies["service"]):
    frames.append(sqldf(f"SELECT * FROM (SELECT * FROM service_genres_movies ORDER BY service, count DESC) WHERE service == \"{service}\" LIMIT {limit}"))

# Combine all of the previosly created DataFrames
top_by_genre = frames[0].copy()
for frame in frames[1:]:
    top_by_genre = sqldf("SELECT * FROM top_by_genre UNION SELECT * FROM frame")
    
# Re-order DataFrame
top_by_genre = sqldf("SELECT * FROM top_by_genre ORDER BY service, count DESC")
top_by_genre

,service,type,genre,count,total_on_service,percentage_of_total
0,amazon,Movie,Drama,1857,8803,0.210951
1,amazon,Movie,Comedy,1241,8803,0.140975
2,amazon,Movie,Thriller,861,8803,0.097808
3,amazon,Movie,Action,736,8803,0.083608
4,amazon,Movie,Romance,652,8803,0.074066
5,disney,Movie,Family,588,2659,0.221136
6,disney,Movie,Comedy,391,2659,0.147048
7,disney,Movie,Animation,349,2659,0.131252
8,disney,Movie,Adventure,292,2659,0.109816
9,disney,Movie,Fantasy,177,2659,0.066566


In [24]:
# Get top service for each genre
movies_top_each_genre = sqldf("SELECT genre, service as top_service, count, MAX(percentage_of_total) as percentage_of_total FROM service_genres_movies GROUP BY genre")
movies_top_each_genre

,genre,top_service,count,percentage_of_total
0,Action,amazon,736,0.083608
1,Adventure,disney,292,0.109816
2,Animation,disney,349,0.131252
3,Comedy,netflix,1833,0.172323
4,Crime,amazon,451,0.051233
5,Documentary,hbo,322,0.092502
6,Drama,amazon,1857,0.210951
7,Family,disney,588,0.221136
8,Fantasy,disney,177,0.066566
9,History,hbo,85,0.024418


## Ratings

G, PG, PG-13, R, etc. 

This will be used to recommend services based on whether family friendly content is needed.

In [25]:
# Count occurrences of each rating for each service
ratings_by_service = sqldf("SELECT service, type, COUNT(rating) as count, rating FROM movies GROUP BY service, rating")
ratings_by_service = sqldf("SELECT * FROM ratings_by_service WHERE count != 0")

In [26]:
# Get genre counts as percentage of total movies
total_count = []
percentages = []
for index, row in ratings_by_service.iterrows():
    total_count.append(np.sum(sqldf(f"SELECT count FROM ratings_by_service WHERE service == \"{row.service}\""))[0])
    percentages.append(row["count"]/total_count[-1])
    

ratings_by_service["total_on_service"] = total_count
ratings_by_service["percentage_of_total"] = percentages

In [27]:
# Save counted ratings as .csv
ratings_by_service.to_csv("data/modified/ratings_counted.csv", index=False)

### More Rating Exploration

Taking a look at distributions of ratings and what services have the highest proportion of each rating. 

In [28]:
# Get the top `limit` ratings on each service
limit = 5
frames = []
# For each service, get `limit` most frequent ratings
for service in np.unique(ratings_by_service["service"]):
    frames.append(sqldf(f"SELECT * FROM (SELECT * FROM ratings_by_service ORDER BY service, count DESC) WHERE service == \"{service}\" LIMIT {limit}"))
    
# Combine all of the previosly created DataFrames
top_by_rating = frames[0].copy()
for frame in frames[1:]:
    top_by_rating = sqldf("SELECT * FROM top_by_rating UNION SELECT * FROM frame")
    
# Re-order DataFrame
top_by_rating = sqldf("SELECT service, count, percentage_of_total, rating FROM top_by_rating ORDER BY service, count DESC")
top_by_rating

,service,count,percentage_of_total,rating
0,amazon,1504,0.334222,R
1,amazon,1491,0.331333,PG-13
2,amazon,702,0.156000,NC-17
3,amazon,501,0.111333,G
4,amazon,197,0.043778,PG
5,disney,444,0.461538,G
6,disney,420,0.436590,PG
7,disney,98,0.101871,PG-13
8,hbo,432,0.363331,R
9,hbo,368,0.309504,PG-13


In [29]:
# Get top service for each rating
movies_top_each_rating = sqldf("SELECT rating, service as top_service, count, MAX(percentage_of_total) as percentage_of_total FROM ratings_by_service GROUP BY rating")
movies_top_each_rating

,rating,top_service,count,percentage_of_total
0,G,disney,444,0.461538
1,NC-17,amazon,702,0.156000
2,NR,amazon,105,0.023333
3,PG,disney,420,0.436590
4,PG-13,amazon,1491,0.331333
5,R,hulu,385,0.503268
